In [1]:
import warnings
warnings.filterwarnings('ignore')
from PyMacroFin.model import macro_model
import numpy as np
import pandas as pd
import time
import PyMacroFin.utilities as util

def define_model(npoints):
    m = macro_model(name='3trees')

    m.set_endog(['q1','q2' ,'q3'], init=[1,1,1])
    m.set_value(['k1', 'k2', 'k3'], init=[1,1,1])
    m.prices = ['q1', 'q2', 'q3']
    m.set_state(['z','a'])

    m.params.add_parameter('gamma',5)
    m.params.add_parameter('rho',0.05)
    m.params.add_parameter('mu_y1',0.02)
    m.params.add_parameter('sig_y1',0.02)
    m.params.add_parameter('mu_y2',0.05)
    m.params.add_parameter('sig_y2',0.05)
    m.params.add_parameter('mu_y3',0.08)
    m.params.add_parameter('sig_y3',0.08)

    m.equation('z1 = z')
    m.equation('z2 = a')
    m.equation('zl = 1 - z1 - z2')
    m.equation('q1_z1 = d(q1,z)')
    m.equation('q1_z2 = d(q1,a)')
    m.equation('q1_z1z1 = d(q1,z,z)')
    m.equation('q1_z1z2 = d(q1,z,a)')
    m.equation('q1_z2z1 = d(q1,a,z)')
    m.equation('q1_z2z2 = d(q1,a,a)')
    
    m.equation('q2_z1 = d(q2,z)')
    m.equation('q2_z2 = d(q2,a)')
    m.equation('q2_z1z1 = d(q2,z,z)')
    m.equation('q2_z1z2 = d(q2,z,a)')
    m.equation('q2_z2z1 = d(q2,a,z)')
    m.equation('q2_z2z2 = d(q2,a,a)')

    m.equation('q3_z1 = d(q3,z)')
    m.equation('q3_z2 = d(q3,a)')
    m.equation('q3_z1z1 = d(q3,z,z)')
    m.equation('q3_z1z2 = d(q3,z,a)')
    m.equation('q3_z2z1 = d(q3,a,z)')
    m.equation('q3_z2z2 = d(q3,a,a)')


    m.equation('k1_z1 = d(k1,z)')
    m.equation('k1_z2 = d(k1,a)')
    m.equation('k1_z1z1 = d(k1,z,z)')
    m.equation('k1_z1z2 = d(k1,z,a)')
    m.equation('k1_z2z1 = d(k1,a,z)')
    m.equation('k1_z2z2 = d(k1,a,a)')
    
    m.equation('k2_z1 = d(k2,z)')
    m.equation('k2_z2 = d(k2,a)')
    m.equation('k2_z1z1 = d(k2,z,z)')
    m.equation('k2_z1z2 = d(k2,z,a)')
    m.equation('k2_z2z1 = d(k2,a,z)')
    m.equation('k2_z2z2 = d(k2,a,a)')

    m.equation('k3_z1 = d(k3,z)')
    m.equation('k3_z2 = d(k3,a)')
    m.equation('k3_z1z1 = d(k3,z,z)')
    m.equation('k3_z1z2 = d(k3,z,a)')
    m.equation('k3_z2z1 = d(k3,a,z)')
    m.equation('k3_z2z2 = d(k3,a,a)')
    
    m.equation('muz1=mu_y1 - (z1 * mu_y1 + z2 * mu_y2 + zl * mu_y3) + (z1 * sig_y1 + z2 * sig_y2 + zl * sig_y3) * (z1 * sig_y1 + z2 * sig_y2 + zl * sig_y3 - sig_y1)', plot=True, latex=r"\mu^{z_1}")
    m.equation('muz2=mu_y2 - (z1 * mu_y1 + z2 * mu_y2 + zl * mu_y3) + (z1 * sig_y1 + z2 * sig_y2 + zl * sig_y3) * (z1 * sig_y1 + z2 * sig_y2 + zl * sig_y3 - sig_y2)')
    m.equation('sigz1=sig_y1 - (z1 * sig_y1 + z2 * sig_y2 + zl * sig_y3)', plot=True, latex=r"\sigma^{z_1}")
    m.equation('sigz2=sig_y2 - (z1 * sig_y1 + z2 * sig_y2 + zl * sig_y3)')
    m.equation('muz1_ari=muz1 * z1', plot=True, latex=r"{z_1}\mu^{z_1}")
    m.equation('muz2_ari=muz2 * z2')
    m.equation('sigz1_ari=sigz1 * z1', plot=True, latex=r"{z_1}\sigma^{z_1}")
    m.equation('sigz2_ari=sigz2 * z2')
    m.equation('muz3_ari=-(muz1_ari + muz2_ari)')
    m.equation('sigz3_ari=-(sigz1_ari + sigz2_ari)')
    m.equation('muz3=(muz3_ari)/(zl)')
    m.equation('sigz3=(sigz3_ari)/(zl)')
    m.equation('mu_q1=(1)/(q1) * (q1_z1 * muz1_ari + q1_z2 * muz2_ari + (1)/(2) * q1_z1z1 * (sigz1_ari)**(2) + (1)/(2) * q1_z2z2 * (sigz2_ari)**(2) + q1_z1z2 * sigz1_ari * sigz2_ari)')
    m.equation('mu_q2=(1)/(q2) * (q2_z1 * muz1_ari + q2_z2 * muz2_ari + (1)/(2) * q2_z1z1 * (sigz1_ari)**(2) + (1)/(2) * q2_z2z2 * (sigz2_ari)**(2) + q2_z1z2 * sigz1_ari * sigz2_ari)')
    m.equation('mu_q3=(1)/(q3) * (q3_z1 * muz1_ari + q3_z2 * muz2_ari + (1)/(2) * q3_z1z1 * (sigz1_ari)**(2) + (1)/(2) * q3_z2z2 * (sigz2_ari)**(2) + q3_z1z2 * sigz1_ari * sigz2_ari)')
    m.equation('sig_q1=(1)/(q1) * (q1_z1 * sigz1_ari + q1_z2 * sigz2_ari)')
    m.equation('sig_q2=(1)/(q2) * (q2_z1 * sigz1_ari + q2_z2 * sigz2_ari)')
    m.equation('sig_q3=(1)/(q3) * (q3_z1 * sigz1_ari + q3_z2 * sigz2_ari)')
    # m.equation('r = rho + gamma * (z*sig_y1 + (1-z)*sig_y2) - gamma * (1 + gamma) / 2 * ((z*sig_y1)**2 + ((1-z)*sig_y2)**2)', plot=True)
    m.equation('zeta1=gamma * z1 * sig_y1')
    m.equation('zeta2=gamma * z2 * sig_y2')
    m.equation('zeta3=gamma * zl * sig_y3')
    m.equation('mu_k1=muz1 - mu_q1 + sig_q1 * (sig_q1 - sigz1)')
    m.equation('mu_k2=muz2 - mu_q2 + sig_q2 * (sig_q2 - sigz2)')
    m.equation('mu_k3=muz3 - mu_q3 + sig_q3 * (sig_q3 - sigz3)')
    m.equation('sig_k1=sigz1 - sig_q1')
    m.equation('sig_k2=sigz2 - sig_q2')
    m.equation('sig_k3=sigz3 - sig_q3')

    m.endog_equation('q1-z/k1')
    m.endog_equation('q2-a/k2')
    m.endog_equation('q3-zl/k3')
    # m.endog_equation('k1_z * mu_z_ari + 0.5 * k1_zz * sig_z_ari**2 - mu_k1 * k1')
    # m.endog_equation('k2_z * mu_z_ari + 0.5 * k2_zz * sig_z_ari**2 - mu_k2 * k2')
    # m.endog_equation('k1_z * sig_z_ari - sig_k1 * k1')
    # m.endog_equation('k2_z * sig_z_ari - sig_k2 * k2')

    # consider the value function r(x)F(x,t)= u(x) +mu(x)F_x+sig(x)^2/2 F_xx + F_t
    # Note that since there is no value variable, the HJB equations here are not used
    m.hjb_equation('mu','z','muz1_ari')
    m.hjb_equation('mu','a','muz2_ari')
    m.hjb_equation('sig','z','sigz1_ari')
    m.hjb_equation('sig','a','sigz2_ari')
    m.hjb_equation('sig','cross','sigz1_ari*sigz2_ari')

    m.hjb_equation('u','k1',0)
    m.hjb_equation('u','k2',0)
    m.hjb_equation('u','k3',0)
    m.hjb_equation('r','k1',"mu_k1")
    m.hjb_equation('r','k2',"mu_k2")
    m.hjb_equation('r','k3',"mu_k3")


    m.options.ignore_HJB_loop = False
    m.options.max_iter_outer_static = 10
    m.options.min_hjb1 = 2
    m.options.min_hjb2 = 2
    m.options.max_hjb = 5
    m.options.import_guess = False
    m.options.guess_function = False
    m.options.inner_plot = False
    m.options.outer_plot = False
    m.options.final_plot = True
    m.options.n0 = npoints
    m.options.start0 = 0.05
    m.options.end0 = 0.95
    m.options.n1 = npoints
    m.options.start1 = 0.05
    m.options.end1 = 0.95
    m.options.inner_solver = 'newton-raphson'
    m.options.parallel = False
    m.options.return_solution = True
    m.options.save_solution = True
    return m

npoints = 50
tic = time.time()
m = define_model(npoints)
df = m.run()
toc = time.time()
print('elapsed time: {}'.format(toc-tic))

Constructing model...
Model construction completed: 2.87 sec
HJB loop 1 iteration 1 with dt = 0.1
    outer static loop iteration 1 completed with error 4.45e+03
    outer static loop iteration 2 completed with error 0
    outer static loop iteration 3 completed with error 0
    outer static loop iteration 4 completed with error 0
HJB loop 1 iteration 1 completed with error 0.0022 and dt = 0.1


RuntimeError: Traceback (most recent call last):
  File "d:\UofT\Research\PDE-work\continuous-time-eco-models\venv\Lib\site-packages\PyMacroFin\model.py", line 399, in run
    df = outerloop(self,df)
     ^^^^^^^^^^^^^^^^^^
  File "d:\UofT\Research\PDE-work\continuous-time-eco-models\venv\Lib\site-packages\PyMacroFin\outerloop.py", line 34, in outerloop
    df = innerloop(m,df)
         ^^^^^^^^^^^^^^^
  File "d:\UofT\Research\PDE-work\continuous-time-eco-models\venv\Lib\site-packages\PyMacroFin\innerloop.py", line 35, in innerloop
    df.loc[n,:] = inner_static(df.loc[n,:],n,m.name)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\UofT\Research\PDE-work\continuous-time-eco-models\venv\Lib\site-packages\PyMacroFin\innerloop.py", line 102, in inner_static
    raise ValueError("There are nans in the arguments for the jacobian at point {0} and inner static loop iteration {1}.\nThis typically is the result of an infeasible endogenous system at this grid point.\nBest solution